<a href="https://colab.research.google.com/github/kdpark0284/HM_titledescription_filtering/blob/main/FilterTitles_Ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoNLPy Start Up With Google Colab

## Install Dependencies

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
pip install openpyxl
pip install rapidfuzz

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

### Install Mecab (Takes 5min+ usually)

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

## (Test before using KoNLPy) -- Not Neccesary

In [ ]:
%%bash
python3 -c "import jpype; jpype.startJVM(convertStrings=True); print(jpype.isJVMStarted())"

In [ ]:
%%bash
pip3 install pytest
TEST_DIR=$(mktemp -d -t test-XXXXXXXXXX)
git clone https://github.com/konlpy/konlpy --depth=1 $TEST_DIR
python3 -m pytest -v $TEST_DIR

# Extraction Process (Read files, Extract Relevant Words)

## Add User Dictionary for Mecab

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

In [ ]:
!pip install jamo

In [ ]:
from jamo import h2j, j2hcj


def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

def make_user_dic_csv(morpheme_type, word_list, user_dic_file_name):
  file_data = []

  for word, score in word_list:
    jongsung_TF = get_jongsung_TF(word)

    line = f"{word},,,{score},{morpheme_type},*,{jongsung_TF},{word},*,*,*,*,*\n"
    # 단어, left-ID, right-ID, Weight, 품사, 의미분류, 종성유무, 읽기, 타입, 첫번째품사, 마지막품사, 표현
    # https://openuiz.blogspot.com/2018/12/mecab-ko-dic.html

    file_data.append(line)

  with open("./user-dic/user-nnp.csv", 'w', encoding='utf-8') as f:
    for line in file_data:
      f.write(line)

In [ ]:
word_list = [('1TB', 0), ('2TB', 0),  ('미패드',0), ('Wi-Fi',0), ('다크레드',0), ('스타라이트',0), ('그라파이트', 0), ('미드나이트', 0), ('블루에어', 0), ('뮤패드', 0),
              ('미스티라일락', 0), ('보타닉그린',0), ('코튼플라워', 0),('시나모롤',0), ('그레이그린', 0), ('펄 화이트', 0),('블루', 0),('미스트 블루', 0), ('그래비티 그레이',0)]

#  단어 추가 방법: ('원하는단어형태', 0) 를 대괄호 안에 넣어주세요. ( )엔 하나의 단어만 사용해주세요.
#  Method to add a word: Insert ('desired_word_form', 0) inside the bracket. Use only one word inside the parentheses.
#  Example) word_list = [('1TB', 0), ('2TB', 0)]

make_user_dic_csv(morpheme_type="NNP", word_list=word_list, user_dic_file_name='user-nnp.csv')

with open("./user-dic/user-nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

!bash autogen.sh
!make
!sudo make install
!bash tools/add-userdic.sh

## File Upload, Load Dependencies

In [ ]:
from collections import Counter

In [ ]:
import pandas as pd
import re
import openpyxl
import pandas as pd
import io
from collections import Counter
from konlpy.tag import Mecab
from konlpy.utils import pprint
from google.colab import files
from rapidfuzz import fuzz, process

mecab = Mecab()
uploaded_file = files.upload ()

"""
Excel 파일 업로드를 해주세요.
Please upload an Excel file.

"""



In [ ]:
# 업로드된 파일 이름 가져오기
file_name = next(iter(uploaded_file))

# 업로드된 파일을 데이터프레임으로 읽기
df = pd.read_excel(io.BytesIO(uploaded_file[file_name]))

serial_num = df["ATTRIBUTES__MODEL_NUMBER__VALUE__KR"]
title = df["DESCRIPTION"]
attributes = df['TYPE__CODE']

기존에는 단어를 최소단위로 하여 필요한 요소를 추출했지만 띄어쓰기, 포맷등의 문제가 엃혀 필요한 단어를 파이썬에서 읽게 하는것이 힘들었음.

MeCab이라는 한국어 형태소 분석 라이브러리를 사용하여 한국어의 최소단위인 형태소를 기반으로 제목, 용량, 색상을 추출하고자 함.

In [ ]:
!python3 --version

## Filter Lists

In [ ]:
product_title = [
                '갤럭시탭', '갤럭시노트', '갤럭시버즈','아이폰', '갤럭시', '프로', '울트라', '맥스', '아이패드', '에어팟', '플립', '폴드', '뮤패드', 'FE', 'FE+'
                '북', '맥북', 'S', 'Ultra', 'M1', 'M2', 'M3', 'M4', 'M5', 'Pro', 'ProMax', 'A', 'Tab', '맥', '미니', '북', '아이맥', '플러스','Z','+',
                'Galaxy', 'iPhone', 'iPad', 'MacBook', 'Zenbook', '그램', '에어', '스타일', '프로맥스', '미패드', 'Air', '탭', '노트', '버즈', 'AirPods',
                'Flip', 'Fold', 'MuPad', 'FE', 'FE+', 'Lite', '서피스', '고', '울트라북', '스튜디오', '베이직'
                  ]

product_type_dict = {
                      "pc" : "IT",
                      "mobilephone" : "IT",
                      "tablet" : "IT",
                      "laptop" : "IT",
}

type_code_name = {
    'refrigerator': '냉장고',
    'refrigeratorsingledoor': '싱글도어냉장고',
    'refrigeratortwodoor': '투도어냉장고',
    'washingmachine': '세탁기',
    'airconditioner': '에어컨',
    'ricecooker':     '밥솥',
    'vacuumcleaner':  '청소기',
    'dishwasher':     '식기세척기',
    'oven':           '오븐',
    'microwave':      '전자레인지',
    'coffeemachine':  '커피머신',
    'kimchirefrigerator': '김치냉장고',
    'dryer':          '건조기',
    'airpurifier':    '공기청정기',
    'razor':            '면도기',
    'waterpurifier':   '정수기',
    'monitor':          '모니터'
    # 필요한 만큼 더 추가
}

color_list = ["문스톤그레이", "스페이스 그레이", "코스믹 그레이", "내추럴", "카밍 차콜", "카밍그린", "오로라화이트", "잉크웰그레이",
              "스노우화이트", "새틴 화이트", "프라임 실버", "라벤더", "스페이스그레이", "루미다크실버", "미드나이트", "플래티넘실버",
              "스페이스 블랙", "그라파이트", "매트블랙", "사파이어", "스타라이트", "이클립스그레이", "새틴화이트", "보라퍼플",
              "글램화이트", "네이비블루", "메탈릭실버", "화이트실버", "카밍베이지", "페블 그레이", "블랙 메탈", "코타 화이트",
              "코타 퍼플", "크림라벤더", "크림 화이트", "크림화이트", "코타 차콜", "실버글라스", "블랙 크롬메탈", "샤인베이지",
              "에센셜화이트", "카밍그린", "크림레몬", "크림 그레이", "스페이스블랙", "실키크림", "샴페인골드", "실키오트밀", "펄 화이트",
              "크림그레이", "다크그레이", "모닝 블루", "그레이스화이트", "미스티 화이트", "에센셜 블루 그레이", "옥스포드그레이", "미스트 블루", "그래비티 그레이",
              "카밍클레이브라운", "사틴 차콜", "베르사유그레이", "매트블랙", "미스티라일락", "보타닉그린", "코튼플라워", "팬텀블랙", "시나모롤", "크림",
              "라이트블루", "다크레드", "라이트그린", "라임", "핑크골드", "딥퍼플", "화이트 티타늄", "블루 티타늄", "블랙 티타늄", "내추럴 티타늄",
              "실버", "베이지", "블랙", "화이트", "블루", "바이올렛", "옐로우", "그레이", "민트", "그린", "핑크", "퍼플", "차콜", "플래티넘", "옐로", "골드", "레드",
              "Black", "White", "Red", "Blue", "Green", "Yellow", "Pink", "Gray", "Silver", "Spacegray", "그레이그린"]

general_stopwords_list = [ '에듀몰', '해피투게딜', '자급제', '재고보유', '하이마트배송!', 'WQXGA +', '30만원대', '개별구매불가', '본체만구매', '자동취소', '1만','다운로드쿠폰',
                        '하이마트배송', '스태킹키트 포함', '스태킹키트 미포함', '한정판매', '한정수량', '온라인 전용', '1주이상소요/클리어런스', '스태킹/앵글',
                        '셀프관리형', '신모델', '즉시배송', '신상', '앵글설치포함,별도비용발생', '하이마트 설치', '최대36개월무', '사은품 증정', '제주마지막1대', '직렬설치',
                        '추가 다운로드 쿠폰', '국내정발', '빠른출발', '진열', '맞춤출수', '전시', '주연테크x하이메이드', '사내판매', '스태킹키트', '카드 추가',
                        '배송지연', '6개월주기 방문관리형', '익일배송', 'UP가전', '특가', '뭉치면싸다', '1만 다운로드 쿠폰', '1주차', '발송예정', '스태킹설치',
                        '사용안함', '스마트픽',  '미디어', '인버터', '정품OS', '단순배송', '2만원할인', '100원딜', '10대', '한정', '단독!', '단독', '추가 카드',
                        'AppleCare+', '1주이상소요', '**',  'DDR5', 'DDR4', 'DDR6', '시력보호', '최신형', '운영', '스태이션 포함!', '추가 쿠폰', '스태킹', '앵글',
                        '최종321만', '10대 한정판매',  '출고', '안드로이드', '배송시간 소요', '방문관리', 'i5.*?', 'i7.*?', 'win11', '20만원대', '링크!', '키트',
                        '사은품 증정', '2주이상 소요', '동시구매행사용', '1만원할인', '할인', '신학기특가', '키트포함', '키트 포함', 'CPU', 'GPU', 'RTX', 'GTX',
                        '각도조절형', '해피투게딜이벤트', '서랍형', '36개월케어십포함', '개별구매불가,본체만구매-자동취소', '전국무료설치', '8코어', '12코어', '16코어', '10코어',
                        '한정수량', '단종', '소진', '특가', '포토상품평이벤트', '어메이징','3월', '4월', '5월', '6월', '7월', '2주이상소요', '3주이상소요', 'NEW', '가벼운',
                        '8월', '9월', '10월', '11월', '12월', '1월', '2월', '행사', '1등급', '당질저감', '벽걸이조절형', '3년케어십', '초고온수', '기획전', '미포함', '포함',
                        '음성인식', '청정바람', '자가관리', '셀프형', '6개월', '방문주기', '시크릿 !', '최대', '10%', '신모델', '공동구매', '하이마트', '1만 다운로드 쿠폰',
                        '3개월', '창립기획', '단독모델', '스태킹설치포함', '자동먼지비움', '무빙키트' , '공기살균', '쇼핑몰', '배송!', '외장', '외장그래픽',
                        '전자동', '오프라인', '온라인', '전용', '개별구매불가 본체만구매-자동취소', '미세먼지극복', '병렬설치', '비밀', '앵글설치포함']

## Exception functions

In [ ]:
# exceptions.py

def handle_process_text_exceptions(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except AttributeError:
            print("MeCab object 'mecab' is not defined or doesn't have 'morphs' method")
            return "", ""
        except Exception as e:
            print(f"An error occurred in process_text: {str(e)}")
            return "", ""
    return wrapper

def handle_find_storage_exceptions(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"An error occurred in find_storage: {str(e)}")
            return 'N/A', 'N/A'
    return wrapper

## Functions

In [ ]:
def advanced_cleaning(descriptions, stopwords, color_list):
    if isinstance(descriptions, str):
        descriptions = [descriptions]

    stopwords_set = set(stopwords)
    stopwords_pattern = re.compile(r'\b(' + '|'.join(re.escape(sw) for sw in stopwords_set) + r')\b', re.IGNORECASE)

    cleaned_descriptions = []

    for description in descriptions:
        if '점]' in description:
            description = description.split('점]')[-1]

        # 모든 알파벳을 대문자로 변환
        description = description.upper()

        # 괄호 내용 처리
        description, bracket_storage, bracket_color = process_brackets(description, color_list)

        # 일반 텍스트에서 용량 및 색상 추출
        general_storage = find_storage1(description)
        general_color = color_extraction1(description, color_list)

        # 최종 용량 및 색상 결정
        storage = bracket_storage if bracket_storage != 'N/A' else general_storage
        color = bracket_color if bracket_color != 'N/A' else general_color

        # 불용어 및 특수 문자 제거
        description = re.sub(r'[^A-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣+\s²-]', ' ', description)
        description = stopwords_pattern.sub('', description)

        # 추가 정제: 연속된 공백 제거
        description = re.sub(r'\s+', ' ', description).strip()

        cleaned_descriptions.append({
            'cleaned_text': description,
            'storage': storage,
            'color': color
        })

    return cleaned_descriptions if len(cleaned_descriptions) > 1 else cleaned_descriptions[0]

def process_brackets(text, color_list):
    storage = 'N/A'
    color = 'N/A'

    def extract_from_bracket(match):
        nonlocal storage, color
        content = match.group(0)  # 괄호 내용 전체 캡처
        bracket_storage = find_storage1(content)
        bracket_color = color_extraction1(content, color_list)
        if bracket_storage != 'N/A':
            storage = bracket_storage
        if bracket_color != 'N/A':
            color = bracket_color
        return ''  # 괄호 내용과 괄호 기호 제거

    # 괄호 내용 처리
    bracket_pattern = r'[\(\[\{][^\)\]\}]+[\)\]\}]'
    text = re.sub(bracket_pattern, extract_from_bracket, text)

    return text, storage, color

def find_storage1(text):
    if not isinstance(text, str):
        raise TypeError("Input 'text' must be a string")
    storage_patterns = [
        r'(?:\d+\s*[GT]B\s*\+\s*)*(\d+)\s*([GT]B)',
        r'(\d+)\s*([GT]B)',
        r'(\d+)\s*(G)'
    ]
    for pattern in storage_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            storage_value = int(match.group(1))
            storage_unit = match.group(2).upper()
            if storage_unit == 'TB':
                storage_value *= 1024
            if storage_value < 64:
                return 'N/A'
            else:
                return f'{storage_value}GB'
    return 'N/A'

def color_extraction1(text, color_list):
    color_pattern = r'\b(' + '|'.join(re.escape(color) for color in color_list) + r')\b'
    match = re.search(color_pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return 'N/A'

# Attribute call Func
def get_product_type(attributes, product_type_dict):
    def classify_type(attribute):
        if attribute in product_type_dict:
            return product_type_dict[attribute]
        else:
            return 'ETC'

    return attributes.map(classify_type)


# ETC 제목 추출 함수
@handle_process_text_exceptions
def title_ETC(description):
    name = description
    return name


# IT 제목 추출 함수
@handle_process_text_exceptions
def process_text(morphs, title_list):
    if not isinstance(morphs, list):
        raise TypeError("Input 'morphs' must be a list")
    if not isinstance(title_list, list):
        raise TypeError("Input 'title_list' must be a list")

    title = []
    i = 0
    plus_used = False
    while i < len(morphs):
        if morphs[i] in title_list:
            title.append(morphs[i])
            i += 1
            if i < len(morphs) and morphs[i].isdigit():
                title.append(morphs[i])
                i += 1
            if i < len(morphs) and morphs[i] == '+' and not plus_used:
                title.append(morphs[i])
                plus_used = True
                i += 1
        else:
            i += 1

    return ''.join(title).strip()

In [ ]:
def create_dataframe_advanced(df, title, product_type_dict, stopwords, color_list, attribute):
    filtered_text = advanced_cleaning(title, stopwords, color_list)  # Data Cleaning

    extracted_title = []
    extracted_color = []
    extracted_storage = []
    mecab = Mecab()

    # Get product types for all items at once
    product_types = get_product_type(attribute, product_type_dict)

    for text, product_type in zip(filtered_text, product_types):
        if product_type == 'IT':
            mecabmorphs = mecab.morphs(text['cleaned_text'])  # 형태소 추출
            # 제목 형태소 추출
            extracted_title_temp = process_text(mecabmorphs, product_title)
            if len(extracted_title_temp.split()) <= 2:  # 추출된 제목이 2단어 이하인 경우
                extracted_title.append(text['cleaned_text'])  # 정제된 텍스트 사용
            else:
                extracted_title.append(extracted_title_temp)
            extracted_color.append(text['color'])
            extracted_storage.append(text['storage'])
        else:
            extracted_title.append(title_ETC(text['cleaned_text']))
            extracted_color.append(text['color'])
            extracted_storage.append('N/A')

    # 제목에서 색상과 용량 정보 제거
    for i in range(len(extracted_title)):
        if extracted_color[i] != 'N/A':
            extracted_title[i] = extracted_title[i].replace(extracted_color[i], '').strip()
        if extracted_storage[i] != 'N/A':
            extracted_title[i] = extracted_title[i].replace(extracted_storage[i], '').strip()

    # Convert lists to Series
    extracted_title = pd.Series(extracted_title)
    extracted_color = pd.Series(extracted_color)
    extracted_storage = pd.Series(extracted_storage)
    extracted_type = pd.Series(product_types)
    original_title = pd.Series(df["DESCRIPTION"])
    dfattributes = pd.Series(attribute)

    cleaned_df = pd.DataFrame({
        'COLOR': extracted_color,
        'STORAGE': extracted_storage,
        'Type': extracted_type,
        'Classification' : dfattributes,
        'Original': original_title,
        'TITLE': extracted_title
        })

    return cleaned_df

In [ ]:
def custom_ratio(s1, s2, score_cutoff=0):
    # 공백을 제거한 문자열로 비교하되, 원본 문자열은 변경하지 않음
    return fuzz.ratio(s1.replace(" ", ""), s2.replace(" ", ""), score_cutoff=score_cutoff)

def group_products(df, it_threshold=96, etc_threshold=95):
    # IT / ETC 제품간 임계값 설정
    product_names = df['TITLE'].tolist()
    grouped_names = []
    scores = []

    for index, row in df.iterrows():
        threshold = it_threshold if row['Type'] == 'IT' else etc_threshold

        product_name = row['TITLE']
        best_match = None
        best_score = 0

        for grouped_name in grouped_names:
            score = custom_ratio(product_name, grouped_name)
            if score >= threshold and score > best_score:
                best_match = grouped_name
                best_score = score

        if best_match:
            grouped_names.append(best_match)
            scores.append(best_score)
        else:
            grouped_names.append(product_name)
            scores.append(100)

    df['Grouped Name'] = grouped_names
    df['Score'] = scores
    return df

def update_grouped_titles(df):
    # Get unique group names
    unique_group_names = df['Grouped Name'].unique()

    # Create a dictionary to store representative names for each group
    representative_names = {}

    for group_name in unique_group_names:
        # Get rows belonging to the current group
        group = df[df['Grouped Name'] == group_name]

        # Count the frequency of each title within the group
        title_counts = group['TITLE'].value_counts()

        # Select the most frequent title as the representative name
        representative_name = title_counts.index[0]

        # Store the representative name for the current group
        representative_names[group_name] = representative_name

    # Update the 'Grouped Name' column with representative names
    df['Grouped Name'] = df['Grouped Name'].map(representative_names)

    # Calculate the score between original title and representative name
    df['Score'] = df.apply(lambda row: custom_ratio(row['TITLE'], row['Grouped Name']), axis=1)

    # Check if the title has been modified
    df['Modified'] = df['TITLE'] != df['Grouped Name']

    return df



def main(Filtered_DF):
    # Group similar product titles
    grouped_df = group_products(Filtered_DF)

    # Update grouped titles with the most frequent unique title as the representative title
    updated_df = update_grouped_titles(grouped_df)

    return updated_df

In [ ]:
Filtered_DF = create_dataframe_advanced(df, title, product_type_dict, general_stopwords_list, color_list,attributes)
updated_df = main(Filtered_DF)
updated_df.to_excel('Grouping_Titles.xlsx', index=False)
files.download('Grouping_Titles.xlsx')

In [ ]:
updated_df.Classification.unique()

In [ ]:
updated_df.head()


list_for_df = []
for i in list(updated_df.Classification.unique()):
    _df = updated_df[(updated_df['Classification'] == i) & (updated_df['Score'] != 100)].head()
    if not _df.empty:
        list_for_df.append(_df)


# Code Execution

In [ ]:
# files.download('Grouping_Titles.xlsx')

In [ ]:
# pd.concat(list_for_df).to_excel('result.xlsx', index=False)
# files.download('result.xlsx')
